#Evaluate retrieval

In [ ]:
!pip install langchain
!pip install -qU langchain-huggingface
!pip install -qU langchain-groq
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
import os

In [ ]:
#base model to generate dataset
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = " " # <-- Groq API Key
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
#embedding model
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# make sure to wrap them with wrappers
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

langchain_llm = LangchainLLMWrapper(llm)
langchain_embeddings = LangchainEmbeddingsWrapper(embeddings)

## Evaluate RAG pipeline

---

- data_sample is sample dataset.  we have to create our own dataset to evaluate RAG - let us call it rag_eval_dataset

- we already have a Q&A dataset - let us call it LLM Dataset


- The vector database will be given all the opinions used in generating these questions in LLM dataset.
  - Select some questions from LLM Dataset. They will be questions in rag_eval_dataset
  - Give these questions to RAG, The retrieved k results will be context in rag_eval_dataset.
  -  The answer column of Q&A dataset will serve the ground truth.

In [ ]:
## dataset- question, context-retrieved from rag pipeline, ground_truth_ground truth answer
## for a particular question there can be multiple retrivals, separated by ','


In [ ]:
## question answer opinion

In [ ]:
from datasets import Dataset
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)

In [ ]:
dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth'],
    num_rows: 2
})

In [ ]:
from ragas.metrics import (
    context_recall,
    context_precision,
)
from ragas import evaluate

result = evaluate(
    dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
    llm=langchain_llm,
    embeddings=langchain_embeddings
)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df = result.to_pandas()
df.head()

,question,contexts,ground_truth,context_precision,context_recall
0,When was the first super bowl?,[The First AFL–NFL World Championship Game was...,"The first superbowl was held on January 15, 1967",1.0,1.0
1,Who won the most super bowls?,"[The Green Bay Packers...Green Bay, Wisconsin....",The New England Patriots have won the Super Bo...,0.0,0.0
